In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,MinMaxScaler, OrdinalEncoder
import random
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings("ignore")

# fechas
from datetime import datetime

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# algoritmos de clasificación
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

# algoritmos de regresión
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

# métricas
import sklearn.metrics as metrics

# hiperparametrización
from sklearn.model_selection import GridSearchCV

# TensorFlow y tf.keras
import tensorflow as tf
from tensorflow import keras

import xgboost as xgb

from numpy import random
random.seed(42)

from sklearn.model_selection import train_test_split

from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Ruta al archivo Parquet 
ruta_archivo = "C:/Users/Herre/Desktop/TFM/Nuclio_EasyMoney/0_Data/df_encoding.csv"
# Leer el archivo Parquet 
df_segmentacion_1 = pd.read_csv(ruta_archivo)

# Crear Nuevas Features

El objetivo de crear nuevas features es para poder hacer una mejor segmentación de los clientes.

**number_of_products:** Número de productos financieros que el cliente tiene con el banco (si tienes una columna que lo indique).

Esta columna podría representar el número total de productos financieros que un cliente tiene con el banco. A partir de las columnas disponibles, podemos considerar cualquier columna que indique un producto financiero (por ejemplo, depósitos, préstamos, tarjetas de crédito, etc.) como parte del total de productos.

In [20]:
# 1. Crear 'number_of_products'
# Suponiendo que cada columna de productos (como 'short_term_deposit', 'loans', etc.) tiene un valor de 0 o 1
product_columns = ['loans', 'mortgage', 'funds', 
                   'em_account_pp', 'credit_card', 'pension_plan', 
                   'emc_account', 'debit_card', 'em_account_p', 'em_acount']

df_segmentacion_1['number_of_products'] = df_segmentacion_1[product_columns].sum(axis=1)


# Tabla Clustering

In [27]:
# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_segmentacion_1)
# Perform K-Means clustering
kmeans = KMeans(n_clusters=7, random_state=42)  # Adjust the number of clusters if needed
df_segmentacion_1['cluster'] = kmeans.fit_predict(X_scaled)

In [28]:
# Definir las columnas relevantes para el clustering
summary_columns = ["salary",
  "number_of_products", "loans", "mortgage", "funds", "em_account_pp", "credit_card", "pension_plan", "emc_account", "debit_card", "em_account_p", "em_acount",
  "region_code", "age", "segment", "gender_V"
]

# Inicializar el DataFrame ficha_df
ficha_df = pd.DataFrame()

# Generar los datos resumen para cada columna
for col in summary_columns:
    resumen_data = df_segmentacion_1[["cluster", col]].groupby("cluster").describe().T[1:]
    ficha_df = pd.concat([ficha_df, resumen_data])

# Definir los componentes del multi-índice
out_index = ["Monetario"] * 1 + ["Productos"] * 11 + ["Social"] * 4  # Ajusta esta lista según las columnas seleccionadas
inner_index = summary_columns
estadisticos = ["mean", "std", "min", "25%", "50%", "75%", "max"]

# Crear el nuevo multi-índice
new_multi_index = []
for oi, ii in zip(out_index, inner_index):
    for es in estadisticos:
        new_multi_index.append((oi, ii, es))

# Generar el multi-índice
def generate_multiindex(list_of_tuples, names):
    return pd.MultiIndex.from_tuples(list_of_tuples, names=names)

names = ["Grupo Indicadores", "Indicador", "Estadístico"]
index_ficha = generate_multiindex(new_multi_index, names)

# Establecer el multi-índice en ficha_df
ficha_df.set_index(index_ficha, inplace=True)

# Calcular el tamaño de cada cluster
tamaño_clusters = df_segmentacion_1.groupby("cluster").size().to_frame().T
tamaño_clusters.set_index(generate_multiindex([("General", "Clúster", "Tamaño")], names), inplace=True)

# Concatenar los datos de tamaño con el DataFrame ficha_df
ficha_df = pd.concat([tamaño_clusters, ficha_df])

# Estilizar el DataFrame con un gradiente de fondo
styled_ficha_df = ficha_df.style.background_gradient(cmap='Blues', axis=1)

# Mostrar el DataFrame estilizado
styled_ficha_df